<a href="https://colab.research.google.com/github/a-deriva/asteroide/blob/main/asteroide_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

In [ ]:
plt.style.use('dark_background')
plt.rc('font', size=15)

In [ ]:
data = pd.read_csv('asteroid_data.txt', sep=',')
data

In [ ]:
fig = plt.figure(figsize=[10, 8])
ax = fig.add_subplot(111, projection='polar')
ax.scatter(data['angle [deg]'] * np.pi / 180, data['dist [AU]'], color='#FF9600', s=15)

theta = np.linspace(0, 2 * np.pi, 1000)
r_T = 1

ax.plot(theta, np.repeat(r_T, 1000), color='blue')
ax.scatter(0, 0, marker='o', color='yellow', s=100)

plt.show()

$$\chi^2=\sum_{i=1}^n\left(\dfrac{r_{obs}-r_{modelo}}{\sigma_{obs}}\right)^2$$
$$r_{modelo}=\dfrac{a(1-e^2)}{1+e\cos\theta},\ e=\dfrac{\sqrt{a^2-b^2}}{a}$$

In [ ]:
def chi2_polar(theta_obs, r_obs, r_err, ai_guess, af_guess, bi_guess, bf_guess, steps, precision):
  start = time.time()
  iteration = 0
  while((af_guess - ai_guess) / np.mean([ai_guess, af_guess]) >= precision) and ((bf_guess - bi_guess) / np.mean([bi_guess, bf_guess]) >= precision):
    chi2 = np.zeros(shape = (steps, steps))
    a_tests = np.linspace(ai_guess, af_guess, steps)
    b_tests = np.linspace(bi_guess, bf_guess, steps)
    for i in range(steps):
      for j in range(steps):
        e = np.sqrt(a_tests[i]**2 - b_tests[j]**2) / a_tests[i]
        chi2[i][j] = np.sum(((r_obs - (a_tests[i] * (1 - e**2) / (1 + e * np.cos(theta_obs)))) / r_err)**2)
    a_min_pos = np.where(chi2 == np.min(chi2))[0][0]
    a_value = a_tests[a_min_pos]
    ai_guess = a_value - 0.2 * a_value / (iteration + 1)
    af_guess = a_value + 0.2 * a_value / (iteration + 1)
    b_min_pos = np.where(chi2 == np.min(chi2))[1][0]
    b_value = b_tests[b_min_pos]
    bi_guess = b_value - 0.2 * b_value / (iteration + 1)
    bf_guess = b_value + 0.2 * b_value / (iteration + 1)
    print('Valores ajustados: a = ' + str(a_value) + ', b = ' + str(b_value))
    iteration += 1
  end = time.time()
  print('Tempo de computação: ' + str(end-start) + ' segundos; com ' + str(100*precision) + '%' + ' de precisão, ' + str(iteration) + ' iterações e ' + str(iteration*steps**2) + ' elipses testadas.')
  return [a_value, b_value]

In [ ]:
a, b = chi2_polar(data['angle [deg]'] * np.pi / 180, data['dist [AU]'], data['dist_err [AU]'], 1, 3, .2, 1, 20, .01)

In [ ]:
fig = plt.figure(figsize=[10, 8])
ax = fig.add_subplot(111, projection='polar')
ax.scatter(data['angle [deg]'] * np.pi / 180, data['dist [AU]'], color='#FF9600', s=15)

theta = np.linspace(0, 2 * np.pi, 1000)
r_T = 1

ax.plot(theta, np.repeat(r_T, 1000), color='blue')
ax.scatter(0, 0, marker='o', color='yellow', s=100)

e = np.sqrt(a ** 2 - b ** 2) / a
r = a * (1 - e ** 2) / (1 + e * np.cos(theta))
ax.plot(theta, r, color='red')

e1 = np.sqrt((a + 0.05 * a) ** 2 - (b + 0.05 * b) ** 2) / (a + 0.05 * a) 
r1 = (a + 0.05 * a) * (1 - e1 ** 2) / (1 + e1 * np.cos(theta))
e2 = np.sqrt((a - 0.05 * a) ** 2 - (b - 0.05 * b) ** 2) / (a - 0.05 * a)
r2 = (a - 0.05 * a) * (1 - e2 ** 2) / (1 + e2 * np.cos(theta))

ax.fill_between(theta, r1, r2, alpha = 0.2, color='red')

plt.show()